In [3]:
!pip install folium
!pip install geopy
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from IPython.display import IFrame

def scrape_population_data():
    """
    Scrapes population data from the Worldometers website for the year 2023.

    This function retrieves data on the population and demographic statistics of countries
    from the specified URL. It processes the data into a structured format and saves it as both
    an Excel file ('countries_data.xlsx') and a CSV file ('countries_data.csv').

    The data collected for each country includes:
    - Country name
    - Population for the year 2023
    - Yearly change in population from 2022 to 2023
    - Population density (people per square kilometer)
    - Net number of migrants
    - Fertility rate
    - Median age
    - Urban population percentage

    Returns:
        pd.DataFrame: A DataFrame containing the population data for all countries.
    """
    url = "https://www.worldometers.info/world-population/population-by-country/"
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    rows = soup.find("table", {'id': 'example2'}).find('tbody').find_all('tr')

    countries_list = []

    for row in rows:
        dic = {}
        dic['Country'] = row.find_all('td')[1].text
        dic['Population 2023'] = row.find_all('td')[2].text.replace(',', '')
        dic['Yearly change from 2022 to 2023'] = row.find_all('td')[3].text
        dic['Density P/km^2'] = row.find_all('td')[5].text
        dic['Migrants_net'] = row.find_all('td')[7].text
        dic['Fertility_Rate'] = row.find_all('td')[8].text
        dic['Median_age'] = row.find_all('td')[9].text
        dic['Urban population'] = row.find_all('td')[10].text
        countries_list.append(dic)

    df = pd.DataFrame(countries_list)
    df.to_excel('countries_data.xlsx', index=False, engine='openpyxl')
    df.to_csv('countries_data.csv', index=False)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    return df


In [7]:
def fetch_and_process_api_data():
    """
    Fetches and processes GDP data from the International Monetary Fund (IMF) API for the year 2024.

    This function retrieves GDP growth data for various countries and their corresponding country names
    from the IMF's API. It processes the data to create a structured format and saves it as both 
    a CSV file ('gdp_data.csv') and an Excel file ('gdp_data.xlsx').

    The data collected includes:
    - Country name
    - GDP growth for the year 2024

    Returns:
        pd.DataFrame: A DataFrame containing the GDP growth data for all countries for the year 2024.
    """
    response1 = requests.get("https://www.imf.org/external/datamapper/api/v1/NGDP_RPCH?periods=2024")
    data1 = response1.json()

    response2 = requests.get("https://www.imf.org/external/datamapper/api/v1/countries")
    data2 = response2.json()

    country_names = {code: country_data['label'] for code, country_data in data2['countries'].items()}

    gdp_data = []
    for code, gdp_info in data1['values']['NGDP_RPCH'].items():
        if code in country_names:
            country_name = country_names[code]
            for year, gdp_value in gdp_info.items():
                gdp_data.append((country_name, gdp_value))

    df = pd.DataFrame(gdp_data, columns=['Country', 'GDP_2024'])
    
    df.to_csv('gdp_data.csv', index=False)
    df.to_excel('gdp_data.xlsx', index=False)

    return df


In [9]:
def combine_dataframes():
    """
    Combines population and GDP data from two different data sources.

    This function retrieves population data using the `scrape_population_data` function and GDP 
    growth data using the `fetch_and_process_api_data` function. It identifies common countries 
    present in both datasets and merges the relevant information into a single DataFrame.

    The resulting DataFrame includes:
    - Country name
    - Population data for 2023
    - GDP growth for the year 2024

    Returns:
        pd.DataFrame: A DataFrame containing combined population and GDP data for countries present 
        in both datasets.
    """
    scrape_data = scrape_population_data()
    fetch_data = fetch_and_process_api_data()

    common_countries = set(scrape_data['Country']).intersection(set(fetch_data['Country']))
    
    scrape_data_common = scrape_data[scrape_data['Country'].isin(common_countries)]
    fetch_data_common = fetch_data[fetch_data['Country'].isin(common_countries)]
    
    combined_df = pd.merge(scrape_data_common, fetch_data_common, on='Country', how='inner')
    
    return combined_df


In [8]:
scrape_population_data()

,Country,Population 2023,Yearly change from 2022 to 2023,Density P/km^2,Migrants_net,Fertility_Rate,Median_age,Urban population
0,India,1450935791,0.89 %,488,"-630,830",2.0,28,37 %
1,China,1419321278,-0.23 %,151,"-318,992",1.0,40,66 %
2,United States,345426571,0.57 %,38,"1,286,132",1.6,38,82 %
3,Indonesia,283487931,0.82 %,156,"-38,469",2.1,30,59 %
4,Pakistan,251269164,1.52 %,326,"-1,401,173",3.5,20,34 %
5,Nigeria,232679478,2.10 %,255,"-35,202",4.4,18,54 %
6,Brazil,211998573,0.41 %,25,"-225,510",1.6,34,91 %
7,Bangladesh,173562364,1.22 %,"1,333","-473,362",2.1,26,42 %
8,Russia,144820423,-0.43 %,9,"-178,042",1.5,40,75 %
9,Ethiopia,132059767,2.62 %,132,"30,069",3.9,19,22 %


In [10]:
fetch_and_process_api_data()

,Country,GDP_2024
0,Malawi,3.3
1,Mali,4.0
2,Malta,5.0
3,Mauritania,5.1
4,Mauritius,4.9
5,"Micronesia, Fed. States of",1.1
6,Moldova,2.6
7,Morocco,3.1
8,Namibia,2.6
9,Netherlands,0.6


In [12]:
combine_dataframes()

,Country,Population 2023,Yearly change from 2022 to 2023,Density P/km^2,Migrants_net,Fertility_Rate,Median_age,Urban population,GDP_2024
0,India,1450935791,0.89 %,488,"-630,830",2.0,28,37 %,6.8
1,United States,345426571,0.57 %,38,"1,286,132",1.6,38,82 %,2.7
2,Indonesia,283487931,0.82 %,156,"-38,469",2.1,30,59 %,5.0
3,Pakistan,251269164,1.52 %,326,"-1,401,173",3.5,20,34 %,2.0
4,Nigeria,232679478,2.10 %,255,"-35,202",4.4,18,54 %,3.3
5,Brazil,211998573,0.41 %,25,"-225,510",1.6,34,91 %,2.2
6,Bangladesh,173562364,1.22 %,"1,333","-473,362",2.1,26,42 %,5.7
7,Ethiopia,132059767,2.62 %,132,"30,069",3.9,19,22 %,6.2
8,Mexico,130861007,0.86 %,67,"-104,581",1.9,29,87 %,2.4
9,Japan,123753041,-0.50 %,339,"153,357",1.2,49,93 %,0.9


In [11]:
from geopy.geocoders import Nominatim
import folium
from folium import Marker, Popup, IFrame
from IPython.display import HTML

def create_country_iframes(combined_df):
    """
    Creates interactive iframes for country maps based on their geographical locations.

    This function uses the combined DataFrame containing country data to generate 
    Folium maps for each country. It retrieves the geographical coordinates of each 
    country using the Nominatim geocoder and creates a corresponding map with a marker 
    for the country's location. The maps are stored as HTML iframes in a dictionary.

    Args:
        combined_df (pd.DataFrame): A DataFrame containing country names and associated data.

    Returns:
        dict: A dictionary where the keys are country names and the values are 
              HTML iframes containing the respective maps.
    """
    geolocator = Nominatim(user_agent="my_geocoder")
    country_iframes = {}

    for index, row in combined_df.iterrows():
        country = row['Country']
        location = geolocator.geocode(country)
        
        if location:
            latitude, longitude = location.latitude, location.longitude
            
            country_map = folium.Map(location=[latitude, longitude], zoom_start=4)
            marker = folium.Marker(location=[latitude, longitude], popup=folium.Popup(country))
            marker.add_to(country_map)

            iframe = IFrame(html=country_map._repr_html_(), width='100%', height=500)
            country_iframes[country] = iframe
    
    return country_iframes

def show_population_map(country_data, country_iframes):
    """
    Displays a global map with markers for countries, each linked to an interactive iframe.

    This function takes a DataFrame containing country data and a dictionary of country 
    iframes to create a global Folium map. Each country is marked on the map, and clicking 
    on a marker displays an interactive iframe that contains the country's map.

    Args:
        country_data (pd.DataFrame): A DataFrame containing country names and associated data.
        country_iframes (dict): A dictionary mapping country names to their corresponding 
                                HTML iframes.

    Returns:
        HTML: An HTML representation of the Folium map with markers for each country.
    """
    folium_map = folium.Map(location=[0, 0], zoom_start=2)
    
    geolocator = Nominatim(user_agent="my_geocoder")
    
    for index, row in country_data.iterrows():
        country = row['Country']
        iframe = country_iframes.get(country)
        if iframe:
            location = geolocator.geocode(country)
            if location:
                latitude, longitude = location.latitude, location.longitude
                folium.Marker([latitude, longitude], 
                              popup=folium.Popup(f'<b>{country}</b><br>{iframe}', max_width=700)).add_to(folium_map)
    
    return HTML(folium_map._repr_html_())
